<h2> Exercise 4 - Hypothesis Testing </h2>

We will now use the specifications in the previous example and consider the hypothesis $H_0:\ \beta_1\neq 0$.  This time I'll use a $\Gamma$ distribution for $x$, which is a distribution commonly used to model waiting times.

In [117]:
using Distributions
using PyPlot
using LinearAlgebra
using Printf
using Random

Random.seed!(14389)

d = Normal(0,1);
N = 50;

alpha = 1.7;
beta1 = -2.0;
beta2 = 3.4;

x1 = rand(d,N);
x2 = rand(d,N);
eps = rand(Normal(0.0,5.0),N); # rand(TDist(5),N);

y = alpha .+ beta1*x1 + beta2*x2 + eps;

X = [ones(N) x1 x2];

betaHat = (X'*X)\(X'*y);

println("Parameter estimate: ", betaHat)

Parameter estimate: [1.6554755458805437, -1.9628330664975442, 3.5642814931247035]


We now need to estimate the variance-covariance matrix $\Sigma$ of $(\widehat{\alpha},\widehat{\beta}_1,\widehat{\beta}_2)$,
\begin{align}
\Sigma = (\mathbb{E} \boldsymbol{x}'\boldsymbol{x})^{-1}\mathbb{E}(\epsilon^2)
\end{align}
To do so, we use the sample analogue
\begin{align}
\widehat{\Sigma} = \left[\left(\sum_{j=1}^N\boldsymbol{x}_j'\boldsymbol{x}_j\right)^{-1}\left(\sum_{j=1}^N \widehat{\epsilon}_j^2\right)\right]
\end{align}

In [118]:
epsHat = y - X*betaHat
SigHat = inv(X'*X)*(epsHat'*epsHat)

println("Variance-covariance estimate: ")
display(SigHat)

Variance-covariance estimate: 


3×3 Matrix{Float64}:
  1.83671   0.482321   -0.4893
  0.482321  1.9024      0.0306327
 -0.4893    0.0306327   1.60712

Next, we construct the test statistic $\widehat{t}$ for $H_0:\beta_1=0$ by dividing the parameter estimate by the square root of $\Sigma_{22}$ and applying the bias-correction factor.

In [119]:
betaHat1 = betaHat[2]
stdErr1 = sqrt(SigHat[2,2])/sqrt(N-3)
tHat1 = betaHat1/stdErr1;

println("Test statistic for H0: beta1=0 is ", tHat1, ", with standard error: ", stdErr1)

Test statistic for H0: beta1=0 is -9.75620382539726, with standard error: 0.20118819795337972


We can now compute the $p$ values as the likelihood of obtaining a value at least as large as in absolution value as $\widehat{t}$.  Precisely, if $f$ denotes the probability density function of a Student's $t$ variable with $3$ degrees of freedom, we want to compute
\begin{align}
\int_{-\infty}^{-|\widehat{t}|} f(t)\; dt + \int_{|\widehat{t}|}^\infty f(t)\; dt
\end{align}
This is equivalent to
\begin{align}
F(-|\widehat{t}|) + (1 - F(|\widehat{t}|)
\end{align}
where $F$ is the cummulative distribution function

In [120]:
p = cdf(TDist(3),-abs(tHat1)) + (1-cdf(TDist(3),abs(tHat1)))

println("p value: ", p)

p value: 0.002287924020119388


We therefore fail to reject the null at significance level 0.05 with $\text{N}(0,5)$ errors.  We can also calculate the critical value $c$ such that we require $|\widehat{t}|\geq c$ in order to reject.  Since Student's $t$ distributions are symmetric, this is determined by solving
\begin{align}
\int_{-\infty}^c f(t)\; dt = 0.025\ \ \ \iff\ \ \ c=F^{-1}(0.025)
\end{align}
To access the inverse CDF in Julia, we can use the quintile command.

In [121]:
critVal = quantile(TDist(3),0.025);
println("The critical value for rejecting the null at significance level 0.05 is ", critVal)

The critical value for rejecting the null at significance level 0.05 is -3.182446305283709


For a further demonstration, let's consider how sample size and error distribution affect the results.  First, we'll consider sample size.

In [122]:
Nbig = 100;
x1bigSamp = rand(Normal(0,1),Nbig);
x2bigSamp = rand(Normal(0,1),Nbig);
epsBigSamp = rand(Normal(0,5),Nbig);
yBigSamp = alpha .+ beta1*x1bigSamp + beta2*x2bigSamp + epsBigSamp;

XbigSamp = [ones(Nbig) x1bigSamp x2bigSamp];

betaHatBigSamp = (XbigSamp'*XbigSamp)\(XbigSamp'*yBigSamp)
epsHatBigSamp = yBigSamp - XbigSamp*betaHatBigSamp

SigHatBigSamp = inv(XbigSamp'*XbigSamp)*(epsHatBigSamp'*epsHatBigSamp)
betaHatBigSamp1 = betaHatBigSamp[2]
stdErrBigSamp1 = sqrt(SigHatBigSamp[2,2])/sqrt(Nbig-3)
tHatBigSamp1 = betaHatBigSamp1/(stdErrBigSamp1);

pBigSamp = cdf(TDist(3),-abs(tHatBigSamp1)) + (1-cdf(TDist(3),abs(tHatBigSamp1)))

println("Testing H0: beta1=0")
println("")
println("Sample Size        coeff.     st.dev.     t        se       p")
println("-------------------------------------------------------------------")
@printf "Small              %.2f      %.2f       %.2f     %.2f     %.2f\n" betaHat1 sqrt(SigHat[2,2]) tHat1 stdErr1 p
@printf "Large              %.2f      %.2f       %.2f     %.2f     %.2f" betaHatBigSamp1 sqrt(SigHatBigSamp[2,2]) tHatBigSamp1 stdErrBigSamp1 pBigSamp

Testing H0: beta<0

Sample Size        coeff.     st.dev.     t        se       p
-------------------------------------------------------------------
Small              -1.96      1.38       -9.76     0.20     0.00
Large              -1.67      4.21       -3.92     0.43     0.03

Running the above code multiple times will generally demonstrate that, with the sample sizes fixed but the sample itself varying, you may end up with drastically different results in your test. As expected, however, the larger sample has a better chance of rejecting the null at the 0.05 significance level.

Finally, let's see how increasing the variance of the residuals affects the above results.

In [123]:
sigBig = 7;
x1bigSig = rand(Normal(0,1),N);
x2bigSig = rand(Normal(0,1),N);
epsBigSig = rand(Normal(0,sigBig),N);
yBigSig = alpha .+ beta1*x1bigSig + beta2*x2bigSig + epsBigSig;

XbigSig = [ones(N) x1bigSig x2bigSig];

betaHatBigSig = (XbigSig'*XbigSig)\(XbigSig'*yBigSig)
epsHatBigSig = yBigSig - XbigSig*betaHatBigSig

SigHatBigSig = inv(XbigSig'*XbigSig)*(epsHatBigSig'*epsHatBigSig)
betaHatBigSig1 = betaHatBigSig[2]
stdErrBigSig1 = sqrt(SigHatBigSig[2,2])/sqrt(N-3)
tHatBigSig1 = betaHatBigSig1/(stdErrBigSig1);

pBigSig = cdf(TDist(3),-abs(tHatBigSig1)) + (1-cdf(TDist(3),abs(tHatBigSig1)))

println("Testing H0: beta1=0")
println("")
println("Sample Size        coeff.     st.dev.     t        se       p")
println("-------------------------------------------------------------------")
@printf "Small              %.2f      %.2f       %.2f     %.2f     %.2f\n" betaHat1 sqrt(SigHat[2,2]) tHat1 stdErr1 p
@printf "Large              %.2f      %.2f       %.2f     %.2f     %.2f" betaHatBigSig1 sqrt(SigHatBigSig[2,2]) tHatBigSig1 stdErrBigSig1 pBigSig

Testing H0: beta<0

Sample Size        coeff.     st.dev.     t        se       p
-------------------------------------------------------------------
Small              -1.96      1.38       -9.76     0.20     0.00
Large              -1.63      8.14       -1.38     1.19     0.26

Higher error variance generally makes it harder for the test to reject the null, although it still manages to do so for some samples.